In [2]:
from itertools import chain
from pathlib import Path
from time import sleep
from datetime import datetime

In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.monitor_interval = 0

In [4]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [5]:
import csv
from collections import Counter, namedtuple
from pathlib import Path

In [6]:
url = 'https://www.basketball-reference.com/boxscores/shot-chart/202003100POR.html'

response = requests.get(url, allow_redirects=False, timeout=10)
soup = BeautifulSoup(response.text, 'html')
NBA_reference = soup.body
shot_point = NBA_reference.find('div', id='content').find('div', class_='section_content').find('div', id='shots-POR').find_all('div')
game_date_text = soup.body.find('div', id='wrap').find('div', id='content').h1.text

In [13]:
def get_shotpoint_info():
    shot_point_all_info = []
    for i in range(0, len(shot_point)):
        shot_tip = shot_point[i]['tip']
        shot_loc = shot_point[i]['style'] #get location
        shot_image = shot_point[i].text #get image

        shot_point_list = shot_point[0]['tip'].replace('<br>', ',').split(',')

        #get quarter of every event
        shot_point_quarter = shot_point_list[0].split(' ')[0]

        #get remaining tim
        shot_point_time_str_list = shot_point_list[1].split(' ')
        shot_point_time_str = shot_point_time_str_list[1]

        #get name of player
        shot_point_player_event_draft = shot_point_list[2].split(' ')
        shot_point_player = shot_point_player_event_draft[0]+' '+shot_point_player_event_draft[1]

        #get event
        shot_point_event = str()
        for i in range(2, len(shot_point_player_event_draft) ):
            shot_point_event = shot_point_event+' '+shot_point_player_event_draft[i]
        
        #get the date of the game
#         game_date_text = soup.body.find('div', id='wrap').find('div', id='content').h1.text
        game_date_list = game_date_text.split(',')
        game_date = game_date_list[1]+game_date_list[2]
        
        shot_point_each_info_nodate = [shot_loc, shot_image, shot_point_quarter, shot_point_time_str, shot_point_player, shot_point_event]
        shot_point_each_info_withdate = [game_date, shot_point_each_info_nodate]
        shot_point_all_info.append(shot_point_each_info_withdate)
    return shot_point_all_info
shot_point_all_info = get_shotpoint_info()
shot_point_all_info

[[' March 10 2020',
  ['top:57px;left:470px;',
   '●',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:46px;left:263px;',
   '●',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:139px;left:20px;',
   '×',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:236px;left:146px;',
   '×',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:34px;left:242px;',
   '●',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:249px;left:107px;',
   '●',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:42px;left:255px;',
   '×',
   '1st',
   '11:30.0',
   'Trevor Ariza',
   ' made 3-pointer from 23 ft']],
 [' March 10 2020',
  ['top:27px;left:240px;',
   '●',
   '1st',
  

In [16]:
import sys

team_list_all = ['BOS', 'BNK', 'NY', 'PHI', 'TOR', 'CHI', 'CLE', 'DET', 'IND', 'MIL', 'ATL', 'CHA', 'MIA', 'ORL', 'WAS', 'DAL', 'HOU', 'MEM', 'NO', 'SA', 'DEN', 'MIN', 'OKC', 'POR', 'UTA', 'GS', 
             'LAC', 'LAL', 'PHX', 'SAC']
team_west = ['DEN', 'MIN', 'POR', 'SEA', 'UTA', 'GS', 'LAC', 'LAL', 'PHO', 'SAC', 'DAL', 'HOU', 'MEM', 'NO', 'SAS']
playoffs_west_2017 = ['GSW', 'SAS', 'HOU', 'LAL', 'UTA', 'OKC', 'MEM', 'POR']
playoffs_east_2017 = ['BOS', 'CLE', 'TOR', 'WAS', 'ALT', 'MIL', 'IND', 'CHI']

shot_point_allinfo_list = []
None_list = []
error_list = []

for i in range(16, 17):
    year = str(i)
    for j in range(1, 13):
        if j<10:
            month = '0'+str(j)
        else:
            month = str(j)
        if j>8 or j<4:
            team_list = team_west
        elif j>=4 and j<6:
            team_list = playoffs_west_2017
        else:
            team_list = playoffs_east_2017
        for k in range(1, 32):
            if k<10:
                date = '0'+str(k)
            else: 
                date = str(k)
            for g in team_list:   
                url = 'https://www.basketball-reference.com/boxscores/shot-chart/20{year}{month}{date}0{g}.html'.format(year=year, month=month, date=date, g=g)
                try:
                    response = requests.get(url, allow_redirects=False, timeout=10)
                    soup = BeautifulSoup(response.text, 'html')
                    NBA_reference = soup.body
                    shot_point = NBA_reference.find('div', id='content').find('div', class_='section_content').find('div', id='shots-POR').find_all('div')
                    game_date_text = soup.body.find('div', id='wrap').find('div', id='content').h1.text
                    shot_point_all_info = get_shotpoint_info()
                    shot_point_allinfo_list.append(shot_point_all_info)  
#                     print(shot_point_all_info)
#                     None_list = None_list.append(soup)
                
                except:
                    #error_list = error_list.append('Unexpected error')
                    print("Unexpected error:", sys.exc_info()[1])

Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has no attribute 'find_all'
Unexpected error: 'NoneType' object has 

In [ ]:
shot_point_allinfo_list[2]

In [22]:
df = pd.DataFrame(shot_point_allinfo_list)

In [62]:
final_list = []
final_list_2 = []
for i in shot_point_allinfo_list:
     for j in i:
        final_list.append(j[0])
        for k in j[1]:
            final_list.append(k)
            final_list_2 = final_list_2+[final_list]

In [ ]:
out_list = []
out_list_all = []
for i in final_list_2:
    for j in range(0, len(i)):
        if j%7 == 0:
            out_list_all = out_list_all+out_list
            out_list = []
            out_list.append(i[j])
        else:
            out_list.append(i[j])
            
        
            
        

In [ ]:
df = pd.DataFrame(final_list_2)
df.head()

In [ ]:
PARENT_DIR = Path.cwd().parent
OUTPUTS_DIR = PARENT_DIR / 'output'
IMAGES_DIR = PARENT_DIR / 'images'

In [ ]:
# PROJECT_DIR = Path.cwd().parent 
# DATA_DIR = PROJECT_DIR / 'data' / 'scraped' / 'teamgamelogs'
# DATA_DIR.mkdir(exist_ok=True, parents=True)
# OUTPUT_DIR = PROJECT_DIR / 'data' / 'prepared'
# OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
CSVFILE = OUTPUTS_DIR.joinpath('Shot_point_info.csv')
df.to_csv(CSVFILE, index=False, quoting=csv.QUOTE_ALL)

In [15]:
for i in range(16, 17):
    year = str(i)
    for j in range(1, 3):
        if j<10:
            month = '0'+str(j)
        else:
            month = str(j)
        if j>8 or j<4:
            team_list = team_west
        elif j>=4 and j<6:
            team_list = playoffs_west_2017
        else:
            team_list = playoffs_east_2017
        for k in range(1, 32):
            if k<10:
                date = '0'+str(k)
            else: 
                date = str(k)
            for g in team_list:   
                url = 'https://www.basketball-reference.com/boxscores/shot-chart/20{year}{month}{date}0{g}.html'.format(year=year, month=month, date=date, g=g)
                print(url)

https://www.basketball-reference.com/boxscores/shot-chart/201601010DEN.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010MIN.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010POR.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010SEA.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010UTA.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010GS.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010LAC.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010LAL.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010PHO.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010SAC.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010DAL.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010HOU.html
https://www.basketball-reference.com/boxscores/shot-chart/201601010MEM.html
https://www.b

In [ ]:
                    NBA_reference = soup.body
                    shot_point = NBA_reference.find('div', id='content').find('div', class_='section_content').find('div', id='shots-POR').find_all('div')
                    game_date_text = soup.body.find('div', id='wrap').find('div', id='content').h1.text
                    shot_point_all_info = get_shotpoint_info()
#                 shot_point_allinfo_list.append(shot_point_all_info)  
                    print(shot_point_all_info)